In [129]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import timm
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import random

# Preparing data

In [150]:
# Import token data
df = pd.read_csv('data/WIFUSDT6mo5m042.csv')

# Remove values where IFFT Slope is 0
df = df[40:].reset_index(drop=True)

print("Hit Stop-Loss (-2%):", (df['Direction'] == 0).sum())
print("Hit Target (+4%):", (df['Direction'] == -1).sum())
print("Hit Target (+4%):", (df['Direction'] == 1).sum())

# Manual Train-Test Split
train_proportion = int(len(df) * 0.8)
df_train = df[:train_proportion]
df_test = df[train_proportion:]

# Output length of training and testing data
print('Training Data:', len(df_train))
print('Testing Data:', len(df_test))

df_train.columns

Hit Stop-Loss (-2%): 12577
Hit Target (+4%): 19730
Hit Target (+4%): 19759
Training Data: 41652
Testing Data: 10414


Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Close Eval', 'IFFT',
       'IFFT Slope Short', 'IFFT Slope Mid', 'IFFT Slope Long', 'ATR',
       'SuperTrend', 'RSI', 'CCI', 'Volatility', 'OBV', 'CDL_DOJI',
       'CDL_HAMMER', 'CDL_ENGULFING', 'CDL_HANGINGMAN', 'CDL_SHOOTINGSTAR',
       'CDL_MORNINGSTAR', 'CDL_EVENINGSTAR', 'Price Change Short',
       'Price Change Mid', 'Price Change Long', 'Direction'],
      dtype='object')

In [151]:
# Train test split
y_train = df_train['Direction']
y_test =  df_test['Direction']
X_train = df_train.drop(columns=['Direction', 'Open', 'High', 'Low'])
X_test = df_test.drop(columns=['Direction', 'Open', 'High', 'Low'])

#X_train.drop('Close Eval',axis = 1,inplace = True)
#X_test.drop('Close Eval',axis = 1,inplace = True)

# Original labels: -1 → down, 0 → flat, 1 → up
# Map to: 0 → down, 1 → flat, 2 → up
label_map = {-1: 0, 0: 1, 1: 2}
y_train = y_train.map(label_map)
y_test = y_test.map(label_map)

In [152]:
# Define continuous columns to be zscored
''''
continuous_cols = ['Open', 'High', 'Low', 'Close', 'Volume', 'IFFT', 'IFFT Slope Short','IFFT Slope Mid', 'IFFT Slope Long', 'ATR', 'SuperTrend', 
                   'RSI', 'CCI', 'Volatility', 'OBV', 'Price Change Short', 'Price Change Mid', 'Price Change Long']
'''
continuous_cols = ['Close', 'Volume', 'IFFT Slope Short', 'IFFT Slope Mid', 'IFFT Slope Long', 'ATR', 'SuperTrend','RSI', 'Volatility', 'OBV']

# Standard scaler
scaler = StandardScaler()

# ZScore continuous training and testing data
X_train[continuous_cols] = scaler.fit_transform(X_train[continuous_cols])
X_test[continuous_cols] = scaler.fit_transform(X_test[continuous_cols])

X_train.shape

(41652, 23)

In [153]:
# Create windowed sequences
def create_sequences(X, y, window_size):
    Xs, ys = [], []
    for i in range(len(X) - window_size):
        Xs.append(X.iloc[i:i+window_size].values)
        ys.append(y.iloc[i + window_size])  # Predict based on full window
    return np.array(Xs), np.array(ys)

window_size = 20  # e.g. 20 past steps (100 minutes if 5m data)
X_train_seq, y_train_seq = create_sequences(X_train, y_train, window_size)
X_test_seq, y_test_seq = create_sequences(X_test, y_test, window_size)

X_train_seq.shape

(41632, 20, 23)

In [154]:
# Define PyTorch Dataset for sequences
class cryptoDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)  # Use float32 for regression

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [155]:
# Create dataset instances
train_dataset = cryptoDataset(X_train_seq, y_train_seq)
test_dataset = cryptoDataset(X_test_seq, y_test_seq)

batch_size = 32 # 32 for more stable gradient updates 128 for faster training if GPU can handle
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Building Model

In [156]:
# Attention layer definition
# Attention assigns weights to each past time step based on how relevant it is
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        self.attention_weights = nn.Linear(hidden_size, 1)

    def forward(self, lstm_outputs):
        # lstm_outputs: [batch, seq_len, hidden_size]
        scores = self.attention_weights(lstm_outputs)  # [batch, seq_len, 1]
        weights = torch.softmax(scores, dim=1)          # normalize over seq_len
        weighted = (lstm_outputs * weights).sum(dim=1)  # weighted sum: [batch, hidden_size]
        return weighted

In [157]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_size=25, hidden_size=128, num_layers=2, num_classes=3):
        super().__init__()

        # Save parameters for use elsewhere (e.g., creating hidden states)
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # Define the LSTM layer
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout = 0.1)

        # Attention
        self.attention = Attention(hidden_size)

        # Dropout (randomly turns off some neurons during training, which prevents the model from relying too heavily on any one feature and helps reduce overfitting)
        self.dropout = nn.Dropout(p=0.1)

        # Output layer maps from LSTM hidden size to number of output classes
        self.output_layer = nn.Linear(hidden_size, num_classes)

    def forward(self, X):

        # Optionally initialize hidden and cell states
        hidden_state = torch.zeros(self.num_layers, X.size(0), self.hidden_size).to(X.device)
        cell_state = torch.zeros(self.num_layers, X.size(0), self.hidden_size).to(X.device)

        # LSTM forward pass
        out, _ = self.lstm(X, (hidden_state, cell_state))  # out: [batch, 128, hidden_size]

        attn_out = self.attention(out) # [batch, hidden_size]

        # Use the output from the final time step
        dropped = self.dropout(attn_out)
        out = self.output_layer(dropped) # [batch, num_classes]
        return out

In [158]:
input_size = X_train.shape[1]
print(input_size)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device, '\n')

model = LSTMClassifier(input_size=input_size).to(device)
print(model)

23
Using device: cuda 

LSTMClassifier(
  (lstm): LSTM(23, 128, num_layers=2, batch_first=True, dropout=0.1)
  (attention): Attention(
    (attention_weights): Linear(in_features=128, out_features=1, bias=True)
  )
  (dropout): Dropout(p=0.1, inplace=False)
  (output_layer): Linear(in_features=128, out_features=3, bias=True)
)


In [159]:
num_epochs = 50
learning_rate = 1e-3

loss_func = nn.CrossEntropyLoss()

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Learning rate scheduler: Reduce LR when validation loss plateaus
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

# Training loop

In [160]:
#for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

In [161]:
if __name__ == "__main__":

    # For saving best model
    best_val_acc = 0.0

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for batch_x, batch_y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)

            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = loss_func(outputs, batch_y)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * batch_x.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == batch_y).sum().item()
            total += batch_y.size(0)

        epoch_loss = running_loss / total
        epoch_acc = correct / total

        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for val_x, val_y in test_loader:
                val_x, val_y = val_x.to(device), val_y.to(device)
                val_outputs = model(val_x)
                loss = loss_func(val_outputs, val_y)
                val_loss += loss.item() * val_x.size(0)
                _, val_predicted = torch.max(val_outputs, 1)
                val_correct += (val_predicted == val_y).sum().item()
                val_total += val_y.size(0)

        val_loss /= val_total
        val_acc = val_correct / val_total

        # Step the LR scheduler
        scheduler.step(val_loss)

        # Save the best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), "best_model.pth")

        print(f"Epoch {epoch+1}/{num_epochs} | "
            f"Train Loss: {epoch_loss:.4f} | Train Acc: {epoch_acc:.4f} | "
            f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

        # Print sample predictions
        print("True labels: ", batch_y[:10].tolist())
        print("Predicted   :", predicted[:10].tolist())

Epoch 1/50:   0%|          | 0/1301 [00:00<?, ?it/s]

Epoch 1/50 | Train Loss: 1.0806 | Train Acc: 0.3637 | Val Loss: 1.0606 | Val Acc: 0.3717
True labels:  [0, 0, 0, 0, 0, 1, 2, 1, 0, 0]
Predicted   : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


Epoch 2/50:   0%|          | 0/1301 [00:00<?, ?it/s]

Epoch 2/50 | Train Loss: 1.0716 | Train Acc: 0.3544 | Val Loss: 1.0572 | Val Acc: 0.3725
True labels:  [0, 0, 0, 0, 0, 1, 2, 1, 0, 0]
Predicted   : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


Epoch 3/50:   0%|          | 0/1301 [00:00<?, ?it/s]

...

Epoch 48/50 | Train Loss: 0.6766 | Train Acc: 0.6949 | Val Loss: 1.4860 | Val Acc: 0.3830
True labels:  [0, 0, 0, 0, 0, 1, 2, 1, 0, 0]
Predicted   : [0, 2, 0, 0, 0, 0, 0, 0, 0, 0]


Epoch 49/50:   0%|          | 0/1301 [00:00<?, ?it/s]

Epoch 49/50 | Train Loss: 0.6766 | Train Acc: 0.6954 | Val Loss: 1.4860 | Val Acc: 0.3829
True labels:  [0, 0, 0, 0, 0, 1, 2, 1, 0, 0]
Predicted   : [1, 0, 0, 0, 0, 0, 0, 0, 1, 0]


Epoch 50/50:   0%|          | 0/1301 [00:00<?, ?it/s]

Epoch 50/50 | Train Loss: 0.6777 | Train Acc: 0.6948 | Val Loss: 1.4861 | Val Acc: 0.3830
True labels:  [0, 0, 0, 0, 0, 1, 2, 1, 0, 0]
Predicted   : [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [162]:
from sklearn.metrics import classification_report

# Assuming val_y_true and val_y_pred are your true and predicted labels for the validation/test set
val_y_true = []  # collect these from your validation loop
val_y_pred = []

model.eval()
with torch.no_grad():
    for val_x, val_y in test_loader:
        val_x, val_y = val_x.to(device), val_y.to(device)
        outputs = model(val_x)
        _, preds = torch.max(outputs, 1)
        val_y_true.extend(val_y.cpu().numpy())
        val_y_pred.extend(preds.cpu().numpy())

print(classification_report(val_y_true, val_y_pred, digits=4))

              precision    recall  f1-score   support

           0     0.4092    0.3608    0.3835      3999
           1     0.2590    0.1979    0.2244      2329
           2     0.4082    0.5108    0.4538      4066

    accuracy                         0.3830     10394
   macro avg     0.3588    0.3565    0.3539     10394
weighted avg     0.3752    0.3830    0.3754     10394



In [163]:
up_true = 0
up_false = 0
down_true = 0
down_false = 0
none_true = 0
none_false = 0

for i in range(len(val_y_true)):
    
    if val_y_true[i] == 0:
        if val_y_true[i] == val_y_pred[i]:    
            down_true += 1
        else:
            down_false += 1

    elif val_y_true[i] == 1:
        if val_y_true[i] == val_y_pred[i]:    
            none_true += 1
        else:
            none_false += 1

    elif val_y_true[i] == 2:
        if val_y_true[i] == val_y_pred[i]:    
            up_true += 1
        else:
            up_false += 1

print('0: ', round(down_true/(down_true + down_false), 2), down_true, '/', down_false)
print('1: ', round(none_true/(none_true + none_false), 2), none_true, '/', none_false)
print('2: ', round(up_true/(up_true + up_false), 2), up_true, '/', up_false)


0:  0.36 1443 / 2556
1:  0.2 461 / 1868
2:  0.51 2077 / 1989


In [164]:
#X_test['True'] = val_y_true
#X_test['Pred'] = val_y_pred
y_test_copy = y_test[20:]

print(len(y_test_copy), len(val_y_true))
val_y_true

count = 0
for i in range(len(val_y_true)):
    if int(y_test_copy.iloc[i]) != int(val_y_true[i]):
        count += 1

count

10394 10394


0

In [165]:
X_test_eval = X_test[20:]
X_test_eval['True'] = val_y_true
X_test_eval['Pred'] = val_y_pred
X_test_eval

<ipython-input-165-debeb36c38c7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_eval['True'] = val_y_true
<ipython-input-165-debeb36c38c7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_eval['Pred'] = val_y_pred


,Close,Volume,Close Eval,IFFT,IFFT Slope Short,IFFT Slope Mid,IFFT Slope Long,ATR,SuperTrend,RSI,...,CDL_ENGULFING,CDL_HANGINGMAN,CDL_SHOOTINGSTAR,CDL_MORNINGSTAR,CDL_EVENINGSTAR,Price Change Short,Price Change Mid,Price Change Long,True,Pred
41672,0.140586,0.760947,0.857,-0.147468,0.737457,1.185985,1.862530,1.011215,0.088196,0.403889,...,0,0,0,0,0,2.316937,-22.714261,-6.403218,1,0
41673,0.075294,0.321899,0.852,-0.149749,0.228047,1.062358,1.778243,0.958362,0.271390,-0.159636,...,0,0,0,0,0,6.196581,-19.290505,-2.854224,0,0
41674,0.101456,0.085442,0.854,-0.151999,-0.303415,0.961813,1.686114,1.001969,0.253576,0.064826,...,0,0,0,0,0,2.272411,-18.981093,-0.000000,0,0
41675,0.101456,-0.364934,0.854,-0.154233,-0.803257,0.858250,1.574302,0.599333,0.243075,0.064826,...,0,0,0,0,0,5.454543,-17.438969,3.051172,0,0
41676,0.153599,1.071738,0.858,-0.155273,-1.086613,0.789695,1.472651,0.516722,0.243075,0.511493,...,0,0,0,0,0,5.602734,-18.334670,-2.960832,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52061,-0.029975,-2.788366,0.844,-0.171217,-0.650863,-0.352519,-0.246379,-0.911330,0.025749,-0.259319,...,0,0,0,0,0,2.136767,-1.379548,59.292960,1,2
52062,0.009617,-0.404173,0.847,-0.170560,-0.496538,-0.333947,-0.253663,-0.861867,0.025749,0.380360,...,0,0,0,0,0,1.440775,-2.092065,62.746403,1,2
52063,0.022783,-0.671661,0.848,-0.169367,-0.214072,-0.263586,-0.210645,-0.908912,0.025749,0.570835,...,0,0,0,0,0,0.719963,-5.436513,53.250291,1,2
52064,-0.003565,-0.649231,0.846,-0.168501,0.047747,-0.193965,-0.171443,-0.860213,0.025749,0.120516,...,-100,0,0,0,-100,-0.702257,-2.755830,57.069972,1,2


In [166]:
up_true = 0
up_false = 0
down_true = 0
down_false = 0
none_true = 0
none_false = 0

for i in range(len(X_test_eval['True'])):

    if X_test_eval['IFFT Slope Short'].iloc[i] < -0.25 and X_test_eval['IFFT Slope Long'].iloc[i] < 0:
        if X_test_eval['True'].iloc[i] == 0:
            if X_test_eval['True'].iloc[i] == X_test_eval['Pred'].iloc[i]:    
                down_true += 1
            else:
                down_false += 1

        elif X_test_eval['True'].iloc[i] == 1:
            if X_test_eval['True'].iloc[i] == X_test_eval['Pred'].iloc[i]:   
                none_true += 1
            else:
                none_false += 1

    elif X_test_eval['IFFT Slope Short'].iloc[i] > 0.25 and X_test_eval['IFFT Slope Long'].iloc[i] > 0: 

        if X_test_eval['True'].iloc[i] == 2:
            if X_test_eval['True'].iloc[i] == X_test_eval['Pred'].iloc[i]:    
                up_true += 1
            else:
                up_false += 1

        elif X_test_eval['True'].iloc[i] == 1:
            if X_test_eval['True'].iloc[i] == X_test_eval['Pred'].iloc[i]:   
                none_true += 1
            else:
                none_false += 1

print('0: ', round(down_true/(down_true + down_false), 2), down_true, '/', down_false)
print('1: ', round(none_true/(none_true + none_false), 2), none_true, '/', none_false)
print('2: ', round(up_true/(up_true + up_false), 2), up_true, '/', up_false)

0:  0.37 378 / 657
1:  0.19 212 / 916
2:  0.51 519 / 500
